In [ ]:
#importing other required libraries
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
#from keras import Sequential laura com
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.applications.resnet import ResNet50
# from tensorflow.python.keras.applications.resnet_v2 import 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Activation,Dropout
from tensorflow.keras.utils import to_categorical

import random
import cv2

#Import the libraries
import zipfile
import os

import pandas as pd
import matplotlib.pyplot  as plt
# from PIL import Image
from pathlib import Path
# import imagesize
import numpy as np

import cv2

import os

from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import random

In [ ]:
def save_model(itr, model_num):
  checkpoint_path = "exp_{}/cp_{}.ckp".format(itr, model_num)
  print(checkpoint_path)
  model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_path,
      monitor='val_loss',
      mode='min', ## ? shld be min???
      save_best_only=True)
  return model_checkpoint_callback


def color_segment_function(img_array):
  '''
  Generate a mask image for the images that is compatible to ImageDataGenerator
  :param image: an image read by cv2.imread
  :return result: masked image
  '''
  img_array= np.rint(img_array)
  img_array= img_array.astype('uint8')
  hsv_img= cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
  mask = cv2.inRange(hsv_img, (24, 40, 0), (60, 255, 255))
  result = cv2.bitwise_and(img_array, img_array, mask=mask)
  result= result.astype('float64')
  return result

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

def call_backs(file_name):
  cb = []
  best_cb= tf.keras.callbacks.ModelCheckpoint(file_name, 
                                    monitor='val_loss', 
                                    verbose=1, 
                                    save_best_only=True, 
                                    save_weights_only=False, 
                                    mode='auto', 
                                    save_freq='epoch')
  cb.append(best_cb)
  
  early = EarlyStopping(monitor="val_loss", 
                        mode="min", 
                        patience=15) # probably needs to be more patience
  
  cb.append(early)

  # Reduce LR On Plateau
  lr_reduced = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_delta=1e-5, patience=8, verbose=1)
  
  cb.append(lr_reduced)
  return cb

In [ ]:
def preprocessing_func(im):
    INP_SIZE = (224, 224) 
    kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
    
#     im = cv2.imread(filename)
    im = cv2.resize(im, (INP_SIZE[0] , INP_SIZE[1]))
    im = cv2.filter2D(src=im, ddepth=-1, kernel=kernel)
    return im

os.mkdir('/kaggle/working/data')
os.mkdir('/kaggle/working/data/test')
!cp /kaggle/input/plant-seedlings-classification/test/*.png /kaggle/working/data/test

In [ ]:
def test_preprocessing_func(im):
    im = color_segment_function(im)
    im = preprocessing_func(im)
    return im

In [ ]:
def class_weight_coeff(data_frame):
  class_weight = {}
  index_weight ={}
  for class_name in list(data_frame.species.unique()):
    count = len(train_df[data_frame.species == class_name])
    class_weight[class_name] = 1/count

  norm_factor= np.mean(list(class_weight.values()))

  i = 0
  for class_name in list(data_frame.species.unique()):
    class_weight[class_name]= class_weight[class_name]/norm_factor
    index_weight[i] = class_weight[class_name]
    i = i+1
  return class_weight, index_weight

change this part of code, for the weights

In [ ]:
def files_to_CSV(src_dir, filename):
  '''
  Generate a csv file for train dataset using the directory stucture of
  of images by mapping each image to the labelled ground truth. The csv 
  is then saved in the current working directory.
  :param src_dir: the directory of the target csv
  :param dest_dir: the directory of the CSV being stored
  :param filename: name of the generated csv
  :return: void
  '''
  #listing filenames from source directory
  fnames = os.listdir(src_dir)
  #sorting list in alphabetical order
  fnames.sort()
  #creating a dictionary assign images to ground truths
  img_dict = {}
  for i in range(len(fnames)):
    img_path = os.path.join(src_dir, fnames[i])
    for img_name in os.listdir(img_path):
      #assigning to key value pair of dictionary
      img_dict[img_name] = fnames[i]
      img_df =  pd.DataFrame(img_dict.items(), columns=['file', 'species'])
      img_df.to_csv(str(filename)+".csv", index = False)
  return 

In [ ]:
#Directory of dataset 
cwd_dir = r'/content'
train_dir = r'../input/segmented-individually/segmented_images'
test_dir = r'/content/test'

In [ ]:
#Generate train.csv
files_to_CSV(src_dir= train_dir, filename='Train')

In [ ]:
train_df = pd.read_csv("./Train.csv")
class_weight , index_weight = class_weight_coeff(train_df)
class_weight_df =pd.DataFrame(list(class_weight.items()),columns = ['class_label','class_weight'])
class_weight_df

In [ ]:
def get_model(train_ds):
    # define model
    #Initializing ResNet50
    base_model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = (224,224,3), classes = len(train_ds.class_indices))
    #Adding layers to the ResNet50
    model_resnet=Sequential()
    #Add the Dense layers along with activation and batch normalization
    model_resnet.add(base_model_resnet)
    model_resnet.add(Flatten())
    #Add the Dense layers along with activation and batch normalization
    # model_resnet.add(Dense(1024,activation=('relu'),input_dim=512))
    model_resnet.add(Dense(512,activation=('relu'))) 
    model_resnet.add(Dropout(0.3))
    model_resnet.add(Dense(512,activation=('relu'))) 
    model_resnet.add(Dropout(0.3))
    model_resnet.add(Dense(256,activation=('relu')))
    model_resnet.add(Dropout(.3))
    model_resnet.add(Dense(256,activation=('relu')))
    model_resnet.add(Dropout(.3))
    model_resnet.add(Dense(128,activation=('relu')))
    model_resnet.add(Dropout(.3))
    model_resnet.add(Dense(12,activation=('softmax')))
    return model_resnet

In [ ]:
# ImageDataGenerator
image_generator = ImageDataGenerator(
    validation_split = 0.20,
    rescale = 1.0/255.0,
    horizontal_flip = True,
    vertical_flip=True,
    width_shift_range=0.3,
    height_shift_range=0.3,
    preprocessing_function = preprocessing_func,
    zoom_range=0.1, 
    shear_range = 0.2,
)

image_generator_test = ImageDataGenerator(rescale = 1.0/255.0, preprocessing_function = test_preprocessing_func)

# paramaters
batch_size= 32
epochs=50

# train_dir = '../input/segmented-individually/segmented_images'
train_dir = '../input/segmented-images/segmented_images'
img_size = [224, 224]

train_ds = image_generator.flow_from_directory(
  train_dir,
  shuffle = True,
  target_size=(img_size[0], img_size[1]),
  class_mode = 'categorical',
  batch_size=batch_size,
  subset="training")

val_ds = image_generator.flow_from_directory(
  train_dir,
  shuffle = True,
  target_size=(img_size[0], img_size[1]),
  class_mode = 'categorical',
  batch_size=batch_size,
  subset="validation")

print(len(train_ds.class_indices))




In [ ]:
model_resnet = get_model(train_ds)
#Compiling ResNet50
# learn_rate=.001
# sgd=SGD(lr=learn_rate)
#Compiling ResNet50
learning_rate = 0.00001 # initial learning rate
decay_rate = 0.1
momentum = 0.8

# define the optimizer function
# sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

#Setting optimzer learning rate & decay
amsgrad= keras.optimizers.Adam(lr= learning_rate, amsgrad=True) 

model_resnet.compile(optimizer =amsgrad, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# train data size / batch size
train_steps = train_ds.samples//batch_size
val_steps = val_ds.samples//batch_size

cb = save_model(1,1)
history = model_resnet.fit_generator(
                train_ds,
                steps_per_epoch = train_steps,
                validation_data = val_ds, 
                validation_steps = val_steps,
                epochs = epochs, 
                callbacks=[cb],
                class_weight = index_weight)

In [ ]:
model_resnet.summary()

## plotting of graph

In [ ]:
f,ax=plt.subplots(2,1) #Creates 2 subplots under 1 column
#Training loss and validation loss
ax[0].plot(history.history['loss'],color='b',label='Training Loss')
ax[0].plot(history.history['val_loss'],color='r',label='Validation Loss')
#Training accuracy and validation accuracy
ax[1].plot(history.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(history.history['val_accuracy'],color='r',label='Validation Accuracy')

In [ ]:
img_size = (224, 224)
test_dir = '/kaggle/working/data'

test_generator = image_generator_test.flow_from_directory(
    test_dir,
    target_size=(img_size[0], img_size[1]),
    class_mode = None,
    batch_size=1,
    shuffle=False)

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model_resnet.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_ds.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames
filenames = [i.split('/')[1] for i in filenames]
results=pd.DataFrame({"file":filenames,
                      "species":predictions})
results.to_csv("results.csv",index=False)

## load best model

In [ ]:
best_model = get_model(train_ds)
checkpoint_path = 'exp_1/cp_1.ckp'
checkpoint_dir = os.path.dirname(checkpoint_path)
best_model.load_weights(checkpoint_path)

In [ ]:
img_size = (224, 224)
test_dir = '/kaggle/working/data'

test_generator = image_generator_test.flow_from_directory(
    test_dir,
    target_size=(img_size[0], img_size[1]),
    class_mode = None,
    batch_size=1,
    shuffle=False)

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=best_model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_ds.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames
filenames = [i.split('/')[1] for i in filenames]
results=pd.DataFrame({"file":filenames,
                      "species":predictions})
results.to_csv("results_best.csv",index=False)